In [1]:
import pandas as pd
df = pd.read_csv("C:/Users/joeco/Python/fb-ref-data/data/181220_2021_aggregated_player_data.csv")
df.head()

,team,name,matches,pos,age,total_mins,goals,assists,pens_made,pens_attempted,...,miscontrols,dispossessed,fouls,times_fouled,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost
0,Southampton,Jake Vokins,1,LB,20,66,0,0,0,0,...,0,0,2,0,0,0,0,2,0,0
1,Sheffield United,Rhian Brewster,17,"FW,LW",20,733,0,0,0,0,...,13,14,12,10,0,0,0,40,6,33
2,Aston Villa,Conor Hourihane,4,CM,29,225,1,1,0,0,...,1,0,1,0,0,0,0,12,3,0
3,Wolverhampton Wanderers,Vitinha,12,DM,20,232,0,1,0,0,...,4,4,8,5,0,0,0,15,0,1
4,Sheffield United,Max Lowe,7,WB,23,460,0,0,0,0,...,3,5,10,2,0,0,0,35,5,16


In [2]:
def create_per_90_stat(row,stat_col):
    total_mins = int(row["total_mins"])
    stat = round(float(row[stat_col]),2)
    total_90s = round(float(total_mins/90),2)
    per_90s_stat = round(float(stat/total_90s),2)
    return per_90s_stat

def compare_players(df,player_list,stat_list,per90=False):
    df_reduced = df[["team","name","total_mins"] + stat_list]
    df_final = df_reduced.query("name in @player_list")
    if per90:
        for stat in stat_list:
            df_final[str(stat) + "_p90"] = df_final.apply(create_per_90_stat,args=(stat,),axis=1)
    return df_final

def sum_stats(row,stats_to_sum):
    total = 0
    for stat in stats_to_sum:
        count = float(row[stat])
        total += count
    return int(total)
    
def compare_players_combined_stats(df,player_list,stats_to_sum,col_name,per90=False):
    df2 = df[["team","name","total_mins"] + stats_to_sum]
    df3 = df2.query("name in @player_list")
    df3[col_name] = df3.apply(sum_stats,args=(stats_to_sum,),axis=1)
    df4 = df3.drop(stats_to_sum,axis=1)
    if per90:
        df4[str(col_name) + "_p90"] = df4.apply(create_per_90_stat,args=(col_name,),axis=1)
        df4.sort_values(by=[str(col_name) + "_p90"],ascending=False,inplace=True)
    else:
        df4.sort_values(by=[col_name],ascending=False,inplace=True)
    return df4

In [4]:
player_list = ["Declan Rice","Pierre Højbjerg","Wilfred Ndidi","Scott McTominay","N'Golo Kanté","Yves Bissouma","Kalvin Phillips",
              "Allan","Rodri","Andre-Frank Zambo Anguissa"]
stat_list = ["tackles","interceptions","blocked_passes","through_balls","assisted_shots","dribbled_past","blocked_shots","clearances","errors","fouls"]
compare_players(df,player_list,stat_list)

,team,name,total_mins,tackles,interceptions,blocked_passes,through_balls,assisted_shots,dribbled_past,blocked_shots,clearances,errors,fouls
15,Everton,Allan,1028,36,7,17,0,7,33,6,13,0,21
97,West Ham United,Declan Rice,1980,54,41,34,4,15,19,6,25,1,22
230,Manchester City,Rodri,1632,43,14,20,1,13,26,5,38,1,24
275,Chelsea,N'Golo Kanté,1478,55,37,38,0,13,26,3,18,0,34
341,Leeds United,Kalvin Phillips,1485,48,28,30,3,19,34,10,34,0,21
358,Manchester United,Scott McTominay,1144,26,18,27,0,10,18,9,26,0,28
380,Tottenham Hotspur,Pierre Højbjerg,1800,72,20,22,1,9,37,7,38,0,41
381,Brighton & Hove Albion,Yves Bissouma,1673,59,23,29,3,7,28,5,30,0,37
450,Leicester City,Wilfred Ndidi,862,46,17,25,0,2,15,4,29,0,20
456,Fulham,Andre-Frank Zambo Anguissa,1677,59,41,29,4,19,22,4,17,1,27


In [6]:
player_list = ["Declan Rice","Pierre Højbjerg","Wilfred Ndidi","Scott McTominay","N'Golo Kanté","Yves Bissouma","Kalvin Phillips",
              "Allan","Rodri","Andre-Frank Zambo Anguissa"]
compare_players_combined_stats(df,player_list,["interceptions","tackles","blocked_passes"],"def_total")

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,team,name,total_mins,def_total
275,Chelsea,N'Golo Kanté,1478,130
97,West Ham United,Declan Rice,1980,129
456,Fulham,Andre-Frank Zambo Anguissa,1677,129
380,Tottenham Hotspur,Pierre Højbjerg,1800,114
381,Brighton & Hove Albion,Yves Bissouma,1673,111
341,Leeds United,Kalvin Phillips,1485,106
450,Leicester City,Wilfred Ndidi,862,88
230,Manchester City,Rodri,1632,77
358,Manchester United,Scott McTominay,1144,71
15,Everton,Allan,1028,60


In [7]:
compare_players_combined_stats(df,player_list,["interceptions","blocked_passes"],"ball_intercepts")

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,team,name,total_mins,ball_intercepts
97,West Ham United,Declan Rice,1980,75
275,Chelsea,N'Golo Kanté,1478,75
456,Fulham,Andre-Frank Zambo Anguissa,1677,70
341,Leeds United,Kalvin Phillips,1485,58
381,Brighton & Hove Albion,Yves Bissouma,1673,52
358,Manchester United,Scott McTominay,1144,45
380,Tottenham Hotspur,Pierre Højbjerg,1800,42
450,Leicester City,Wilfred Ndidi,862,42
230,Manchester City,Rodri,1632,34
15,Everton,Allan,1028,24


In [9]:
 player_list = ["Declan Rice","Pierre Højbjerg","Wilfred Ndidi","Scott McTominay","N'Golo Kanté","Yves Bissouma","Kalvin Phillips",
              "Allan","Rodri","Andre-Frank Zambo Anguissa"]
compare_players_combined_stats(df,player_list,["dribbled_past","fouls"],"failed_tackles",per90=True)

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,team,name,total_mins,failed_tackles,failed_tackles_p90
15,Everton,Allan,1028,54,4.73
380,Tottenham Hotspur,Pierre Højbjerg,1800,78,3.90
275,Chelsea,N'Golo Kanté,1478,60,3.65
450,Leicester City,Wilfred Ndidi,862,35,3.65
358,Manchester United,Scott McTominay,1144,46,3.62
381,Brighton & Hove Albion,Yves Bissouma,1673,65,3.50
341,Leeds United,Kalvin Phillips,1485,55,3.33
230,Manchester City,Rodri,1632,50,2.76
456,Fulham,Andre-Frank Zambo Anguissa,1677,49,2.63
97,West Ham United,Declan Rice,1980,41,1.86


In [13]:
player_list = ["Declan Rice","Pierre Højbjerg","Wilfred Ndidi","Scott McTominay","N'Golo Kanté","Yves Bissouma","Kalvin Phillips",
              "Allan","Rodri","Andre-Frank Zambo Anguissa"]
stat_list = ["tackles","dribbled_past","fouls"]
tackle_success_df = compare_players(df,player_list,stat_list)
tackle_success_df

,team,name,total_mins,tackles,dribbled_past,fouls
15,Everton,Allan,1028,36,33,21
97,West Ham United,Declan Rice,1980,54,19,22
230,Manchester City,Rodri,1632,43,26,24
275,Chelsea,N'Golo Kanté,1478,55,26,34
341,Leeds United,Kalvin Phillips,1485,48,34,21
358,Manchester United,Scott McTominay,1144,26,18,28
380,Tottenham Hotspur,Pierre Højbjerg,1800,72,37,41
381,Brighton & Hove Albion,Yves Bissouma,1673,59,28,37
450,Leicester City,Wilfred Ndidi,862,46,15,20
456,Fulham,Andre-Frank Zambo Anguissa,1677,59,22,27


In [16]:
def tackle_success_percent(row):
    tackles = row["tackles"]
    dribbled_past = row["dribbled_past"]
    tackle_success = round( (tackles/(tackles+dribbled_past)*100) , 1)
    return tackle_success

tackle_success_df["tackle_success"] = tackle_success_df.apply(tackle_success_percent,axis=1)
tackle_success_df.sort_values(by="tackle_success",inplace=True)

In [17]:
tackle_success_df

,team,name,total_mins,tackles,dribbled_past,fouls,tackle_success
15,Everton,Allan,1028,36,33,21,52.2
341,Leeds United,Kalvin Phillips,1485,48,34,21,58.5
358,Manchester United,Scott McTominay,1144,26,18,28,59.1
230,Manchester City,Rodri,1632,43,26,24,62.3
380,Tottenham Hotspur,Pierre Højbjerg,1800,72,37,41,66.1
381,Brighton & Hove Albion,Yves Bissouma,1673,59,28,37,67.8
275,Chelsea,N'Golo Kanté,1478,55,26,34,67.9
456,Fulham,Andre-Frank Zambo Anguissa,1677,59,22,27,72.8
97,West Ham United,Declan Rice,1980,54,19,22,74.0
450,Leicester City,Wilfred Ndidi,862,46,15,20,75.4
